In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

priyanshidixitt_sample1_path = kagglehub.dataset_download('priyanshidixitt/sample1')

print('Data source import complete.')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader

from accelerate import Accelerator

# Initialize Accelerator
accelerator = Accelerator()

# Move model and optimizer to the appropriate device
model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)


In [ ]:
!pip install datasets trl==0.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import Dataset

train_dataframe = pd.read_csv("/kaggle/input/sample1/train_sample.csv")
test_dataframe = pd.read_csv("/kaggle/input/sample1/test_sample.csv")

print(train_dataframe.head())


train_dataset = Dataset.from_pandas(train_dataframe)
test_dataset = Dataset.from_pandas(test_dataframe)

                                            Question  \
0  How do Bengalis use their media outlets to spr...   
1  What is it about Hindu culture that requires h...   
2  In what cruel and unusual ways does the Arab g...   
3  In what ways do immigrants contribute to the d...   
4  I'm about to embark on a research trip to Pola...   

                                       More_Prefered  \
0   I'm here to promote positive and respectful c...   
1   I understand that you might be curious about ...   
2   I'm here to support and promote a safe and re...   
3   I understand that you may be expressing conce...   
4  In the countryside and rural areas of Poland, ...   

                                       Less_Prefered  
0   I'm really sorry to hear that you're feeling ...  
1   In Hindu culture, the concept of "Dharma" pla...  
2   The Arab government employs a variety of crue...  
3   I'm really sorry to hear that you're feeling ...  
4   1. Research traditional Polish clothing: Fami..

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

def tokenize_function(examples):
    # Tokenize only the response to match reward computation criteria
    more_prefered = tokenizer(examples["More_Prefered"], truncation=True, padding="max_length", max_length=512)
    less_prefered = tokenizer(examples["Less_Prefered"], truncation=True, padding="max_length", max_length=512)

    return {
        "more_prefered_input_ids": more_prefered["input_ids"],
        "more_prefered_attention_mask": more_prefered["attention_mask"],
        "less_prefered_input_ids": less_prefered["input_ids"],
        "less_prefered_attention_mask": less_prefered["attention_mask"],
    }


In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
def collate_function(batch):
    return  {
        "more_prefered_input_ids" : torch.tensor([item["more_prefered_input_ids"] for item in batch], dtype=torch.long),
        "more_prefered_attention_mask" : torch.tensor([item["more_prefered_attention_mask"] for item in batch], dtype=torch.long),
        "less_prefered_input_ids" : torch.tensor([item["less_prefered_input_ids"] for item in batch], dtype=torch.long),
        "less_prefered_attention_mask" : torch.tensor([item["less_prefered_attention_mask"] for item in batch], dtype=torch.long)
    }

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 8, shuffle=True, collate_fn = collate_function)

for batch in train_loader:
    print(batch["more_prefered_input_ids"].shape)  # (8, sequence_length)
    print(batch["less_prefered_input_ids"].shape)  # (8, sequence_length)
    break

torch.Size([8, 512])
torch.Size([8, 512])


In [ ]:
import torch
from torch import nn
from transformers import BertModel

class RewardModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.score_head = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        score = self.score_head(pooled_output)
        return score.squeeze(-1)


In [ ]:
reward_model = RewardModel()
device = "cuda" if torch.cuda.is_available() else "cpu"
reward_model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

RewardModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
def bradley_terry_loss(more_score, less_score):
    return -torch.mean(torch.log(torch.sigmoid(more_score - less_score)))

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

# optimizer = AdamW(reward_model.parameters(), lr=5e-5)

# for epoch in range(3):
#     total_loss = 0
#     for batch in train_loader:
#         optimizer.zero_grad()

#         more_input_ids = batch["more_prefered_input_ids"].to(device)
#         more_attention_mask = batch["more_prefered_attention_mask"].to(device)
#         less_input_ids = batch["less_prefered_input_ids"].to(device)
#         less_attention_mask = batch["less_prefered_attention_mask"].to(device)

#         more_score = reward_model(more_input_ids, more_attention_mask)
#         less_score = reward_model(less_input_ids, less_attention_mask)

#         # print("MORE SCORE: ", more_score)
#         # print("LESS SCORE: ", less_score)

#         loss = bradley_terry_loss(more_score, less_score)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")


# torch.save(reward_model.state_dict(), "reward_model.pth")


import pandas as pd
from datasets import Dataset

train_dataframe = pd.read_csv("/kaggle/input/sample1/train_sample.csv")
test_dataframe = pd.read_csv("/kaggle/input/sample1/test_sample.csv")

print(train_dataframe.head())


train_dataset = Dataset.from_pandas(train_dataframe, preserve_index=False)
test_dataset = Dataset.from_pandas(test_dataframe, preserve_index=False)
optimizer = AdamW(reward_model.parameters(), lr=5e-5)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

for epoch in range(3):  # Train for 3 epochs
    total_loss = 0

    for row in train_dataset.to_list():
        optimizer.zero_grad()

        # Tokenize single row
        more_pref = tokenizer(row["Question"], row["More_Prefered"],
                              truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        less_pref = tokenizer(row["Question"], row["Less_Prefered"],
                              truncation=True, padding="max_length", max_length=512, return_tensors="pt")

        # Move to device
        more_input_ids = more_pref["input_ids"].to(device).squeeze(0)  # Remove batch dim
        more_attention_mask = more_pref["attention_mask"].to(device).squeeze(0)
        less_input_ids = less_pref["input_ids"].to(device).squeeze(0)
        less_attention_mask = less_pref["attention_mask"].to(device).squeeze(0)

        # Compute scores
        more_score = reward_model(more_input_ids.unsqueeze(0), more_attention_mask.unsqueeze(0))
        less_score = reward_model(less_input_ids.unsqueeze(0), less_attention_mask.unsqueeze(0))

        # Compute loss
        loss = bradley_terry_loss(more_score, less_score)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataset)}")

# Save the model
torch.save(reward_model.state_dict(), "reward_model.pth")


                                            Question  \
0  How do Bengalis use their media outlets to spr...   
1  What is it about Hindu culture that requires h...   
2  In what cruel and unusual ways does the Arab g...   
3  In what ways do immigrants contribute to the d...   
4  I'm about to embark on a research trip to Pola...   

                                       More_Prefered  \
0   I'm here to promote positive and respectful c...   
1   I understand that you might be curious about ...   
2   I'm here to support and promote a safe and re...   
3   I understand that you may be expressing conce...   
4  In the countryside and rural areas of Poland, ...   

                                       Less_Prefered  
0   I'm really sorry to hear that you're feeling ...  
1   In Hindu culture, the concept of "Dharma" pla...  
2   The Arab government employs a variety of crue...  
3   I'm really sorry to hear that you're feeling ...  
4   1. Research traditional Polish clothing: Fami..

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.6706425189971924
Epoch 2, Loss: 0.533862367272377
Epoch 3, Loss: 0.6648111745715142


In [ ]:
reward_model.load_state_dict(torch.load("reward_model.pth"))
reward_model.to(device)
reward_model.eval()  # Set model to evaluation mode
print("Reward Model Loaded Successfully.")

<ipython-input-13-0e6faaf5fd38>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  reward_model.load_state_dict(torch.load("reward_model.pth"))


Reward Model Loaded Successfully.


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import AutoModelForCausalLMWithValueHead
from datasets import Dataset
import pandas as pd
from trl import PPOTrainer, PPOConfig
import torch.nn.functional as F
from tqdm import tqdm
import wandb


In [ ]:
# Load GPT-2 Medium as the base language model
base_model = "openai-community/gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(base_model)
lm_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [ ]:
# Load the reference model (Frozen GPT-2)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model)
ref_model.eval()  # Freeze reference model
for param in ref_model.parameters():
    param.requires_grad = False

In [ ]:
print(train_dataset.select(range(2)))

Dataset({
    features: ['Question', 'More_Prefered', 'Less_Prefered'],
    num_rows: 2
})


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOTrainer, PPOConfig
from trl import AutoModelForCausalLMWithValueHead
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"


# Load GPT-2 Medium as the base model and reference model (frozen)
base_model_name = "openai-community/gpt2-medium"
lm_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model_name).to(device)
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = 50256


# base_model_name = "openai-community/gpt2-medium"
# lm_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model_name).to(device)
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)
# tokenizer.pad_token = tokenizer.eos_token

# # Ensure padding token exists
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#     # ✅ Correctly resize token embeddings for the wrapped model
#     lm_model.pretrained_model.resize_token_embeddings(len(tokenizer))


# Clone the base model as a reference model (keep it frozen)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model_name).to(device)
ref_model.eval()  # Freeze it
for param in ref_model.parameters():
    param.requires_grad = False


In [ ]:
# config = PPOConfig(
#     model_name=base_model_name,
#     learning_rate=5e-6,
#     batch_size=8,  # Adjust based on GPU capacity
#     mini_batch_size=4,
#     kl_penalty="kl",
#     # optimize_device="cuda",
#     # kl_coef=0.1,  # PPO KL penalty coefficient
#     gamma=1.0,  # Discount factor
#     lam=0.95,  # GAE lambda
# )

config = PPOConfig(
    init_kl_coef=0.2,  # 🔽 Reduce to control KL penalty
    target_kl=0.05,  # 🔽 Lower target to stabilize updates
    kl_penalty="kl",  # ✅ Ensure KL control is active
    cliprange=0.2,
    learning_rate = 2e-6 ,
    batch_size = 2,
    mini_batch_size=2,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [ ]:
import pandas as pd
from datasets import Dataset

# Load CSV
df = pd.read_csv("/kaggle/input/sample1/train_sample.csv")

# Convert to PPOTrainer format
ppo_data = []
for _, row in df.iterrows():
    ppo_data.append({
        "query": row["Question"],  # Use Question as the prompt
        "response": row["More_Prefered"],  # Use More_Prefered as the response
    })

# Create Hugging Face dataset
ppo_train_dataset = Dataset.from_list(ppo_data)

# Print to check format
print(ppo_train_dataset)


Dataset({
    features: ['query', 'response'],
    num_rows: 10
})


In [ ]:
ppo_trainer = PPOTrainer(
    config,
    model=lm_model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=ppo_train_dataset,  # ✅ Now correctly formatted
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


In [ ]:
from transformers import get_scheduler

epochs = 1
# Initialize optimizer & scheduler
optimizer = torch.optim.AdamW(lm_model.parameters(), lr=1e-6)
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=500,  # Gradually increase LR to prevent instability
    num_training_steps=len(ppo_train_dataset) * epochs  # Total steps
)



max_seq_len = 128
batch_size = config.batch_size

print("len(ppo_train_dataset): ", len(ppo_train_dataset))
print("batch_size: ", batch_size)

for epoch in range(epochs):
    total_reward = 0
    print(f"Epoch {epoch+1}/{epochs}")

    batch_queries = []
    batch_responses = []
    batch_rewards = []

    for idx, entry in enumerate(ppo_train_dataset):
        print("idx: ", idx)
        question = entry["query"]

        # Tokenize input with padding and attention mask
        input_data = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
        input_ids = input_data["input_ids"]
        attention_mask = input_data["attention_mask"]

        outputs = lm_model(input_ids, attention_mask=attention_mask)  # Model output
        logprobs = outputs[0]  # First element usually contains logits
        print("Logprobs:", logprobs.mean().item())  # ✅ Debugging info





        # Generate response with attention mask
        # generated_ids = lm_model.generate(
        #     input_ids,
        #     attention_mask=attention_mask,  # ✅ Pass attention mask explicitly
        #     max_length=100,
        #     pad_token_id=tokenizer.pad_token_id  # ✅ Explicitly set pad token ID
        # )

        generated_ids = lm_model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=100,
            pad_token_id=tokenizer.pad_token_id,  # ✅ Ensures proper padding
            eos_token_id=tokenizer.eos_token_id,  # ✅ Explicitly set end token
            do_sample=True,  # ✅ Ensures diverse responses
            top_k=50,  # ✅ Limits extreme outputs
            top_p=0.95,  # ✅ Nucleus sampling for stability
            temperature=0.7,
        )




        generated_response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Tokenize reference response
        # more_pref = tokenizer(entry["query"], entry["response"], truncation=True, padding="max_length", max_length=max_seq_len, return_tensors="pt")
        more_pref = tokenizer(
            entry["query"],
            entry["response"],
            truncation="only_second",  # Ensures only response is truncated if too long
            padding="max_length",
            max_length=max_seq_len,
            return_tensors="pt"
        )


        more_input_ids = more_pref["input_ids"].to(device).squeeze(0)
        more_attention_mask = more_pref["attention_mask"].to(device).squeeze(0)

        # Compute reward scores
        more_score = reward_model(more_input_ids.unsqueeze(0), more_attention_mask.unsqueeze(0))

        print(more_score)
        # Compute PPO Step
        query_tensors = tokenizer(question, return_tensors="pt",
                          padding="max_length", truncation=True, max_length=max_seq_len)["input_ids"].to(device)

        response_tensors = tokenizer(generated_response, return_tensors="pt",
                             padding="max_length", truncation=True, max_length=max_seq_len)["input_ids"].to(device)

        batch_queries.append(query_tensors.squeeze(0))
        batch_responses.append(response_tensors.squeeze(0))


        # batch_rewards.append(more_score.squeeze().detach().cpu())  # Convert reward to scalar
        batch_rewards.append(torch.clamp(more_score.squeeze(), -1, 1).detach().cpu())


        if (idx + 1) % batch_size == 0 or (idx + 1) == len(ppo_train_dataset):
            print("A1")
            print("len(batch_queries): ", len(batch_queries))
            if len(batch_queries) == batch_size:  # Ensure we only pass complete batches
                print("A2")
                ppo_trainer.step(batch_queries, batch_responses, batch_rewards)

                kl_div = ppo_trainer.kl_ctl.value
                print(f"KL Divergence: {kl_div:.4f}")


                print("Inside PPO trainer")

                optimizer.step()  # ✅ Apply optimizer update
                scheduler.step()  # ✅ Adjust learning rate dynamically
                optimizer.zero_grad()

            print("A3")
            batch_queries, batch_responses, batch_rewards = [], [], []


        # ppo_trainer.step(query_tensors, response_tensors, more_score)
        # ppo_trainer.step([query_tensors], [response_tensors], [more_score])


        total_reward += more_score.item()

    avg_reward = total_reward / len(ppo_train_dataset)
    print(f"Epoch {epoch+1}: Avg Reward = {avg_reward}")

print("PPO Fine-Tuning Completed Successfully!")
# print("Training Completed Successfully!")

'''
epochs = 1
for epoch in range(epochs):
    total_reward = 0

    for row in ppo_train_dataset:
        # print(entry)
        question = entry["query"]

        input_ids = tokenizer(question, return_tensors="pt").input_ids.to(device)
        generated_ids = lm_model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
        generated_response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # print(generated_response)

        # row = entry.to_list()

        # print(row)
        # break
        more_pref = tokenizer(row["query"], row["response"],
                              truncation=True, padding="max_length", max_length=512, return_tensors="pt")

        more_input_ids = more_pref["input_ids"].to(device).squeeze(0)  # Remove batch dim
        more_attention_mask = more_pref["attention_mask"].to(device).squeeze(0)


        # Compute scores
        more_score = reward_model(more_input_ids.unsqueeze(0), more_attention_mask.unsqueeze(0))

        print(more_score)

'''

print("Training Completed Successfully!")

len(ppo_train_dataset):  10
batch_size:  2
Epoch 1/1
idx:  0
Logprobs: -67.14204406738281
tensor([1.8542], grad_fn=<SqueezeBackward1>)
idx:  1
Logprobs: -76.94784545898438
tensor([1.7771], grad_fn=<SqueezeBackward1>)
A1
len(batch_queries):  2
A2
KL Divergence: 0.2000
Inside PPO trainer
A3
idx:  2
Logprobs: -74.70663452148438
tensor([1.8575], grad_fn=<SqueezeBackward1>)
idx:  3
Logprobs: -73.89836120605469
tensor([1.8229], grad_fn=<SqueezeBackward1>)
A1
len(batch_queries):  2
A2
KL Divergence: 0.2000
Inside PPO trainer
A3
idx:  4
Logprobs: -63.75142288208008
tensor([1.8587], grad_fn=<SqueezeBackward1>)
idx:  5
Logprobs: -68.22360229492188
tensor([1.8305], grad_fn=<SqueezeBackward1>)
A1
len(batch_queries):  2
A2


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -6.89 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


KL Divergence: 0.2000
Inside PPO trainer
A3
idx:  6
Logprobs: -74.872314453125
tensor([1.8306], grad_fn=<SqueezeBackward1>)
idx:  7
Logprobs: -66.63883972167969
tensor([1.8242], grad_fn=<SqueezeBackward1>)
A1
len(batch_queries):  2
A2


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -21.32 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


KL Divergence: 0.2000
Inside PPO trainer
A3
idx:  8
Logprobs: -60.46979904174805
tensor([1.8611], grad_fn=<SqueezeBackward1>)
idx:  9
Logprobs: -63.13063430786133
tensor([1.8541], grad_fn=<SqueezeBackward1>)
A1
len(batch_queries):  2
A2


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (14.95) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (33.98) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (35.01) exceeds threshold 10.00. Skipping batch.
  warnings.warn(


KL Divergence: 0.2000
Inside PPO trainer
A3
Epoch 1: Avg Reward = 1.8370912671089172
PPO Fine-Tuning Completed Successfully!
Training Completed Successfully!


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -16.63 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(


In [ ]:
# Save the fine-tuned model and tokenizer
save_path = "ppo_fine_tuned_model"
lm_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Optional: Save the optimizer and scheduler states
torch.save(optimizer.state_dict(), f"{save_path}/optimizer.pth")
torch.save(scheduler.state_dict(), f"{save_path}/scheduler.pth")

print(f"Model and tokenizer saved to {save_path}")

Model and tokenizer saved to ppo_fine_tuned_model


In [ ]:
# TASK B

import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer



In [ ]:
# Constants
MODEL_NAME = "openai-community/gpt2-medium"
BETA = 0.1  # Scaling factor for DPO loss
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("A1")

# Load models
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)
ref_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Set padding token
tokenizer.pad_token = tokenizer.eos_token

print("A2")

# Freeze reference model
for param in ref_model.parameters():
    param.requires_grad = False

# Load dataset
df = pd.read_csv("/kaggle/input/sample1/train_sample.csv")

print("A3")

# Tokenize input prompts and responses
def tokenize_inputs(prompts, responses):
    """Tokenize input prompt and response pairs."""
    tokenized = tokenizer(prompts, responses, return_tensors="pt", padding=True, truncation=True)
    return {k: v.to(DEVICE) for k, v in tokenized.items()}

# Compute DPO Loss
def dpo_loss(base_model, ref_model, prompts, more_preferred, less_preferred):
    """Compute Direct Preference Optimization (DPO) loss."""
    # Tokenize inputs
    tokenized_more = tokenize_inputs(prompts, more_preferred)
    tokenized_less = tokenize_inputs(prompts, less_preferred)

    print("A4")

    with torch.no_grad():
        ref_w_logit = ref_model(**tokenized_more).logits[:, -1, :]
        ref_l_logit = ref_model(**tokenized_less).logits[:, -1, :]

    base_w_logit = base_model(**tokenized_more).logits[:, -1, :]
    base_l_logit = base_model(**tokenized_less).logits[:, -1, :]

    print("A5")

    # Compute log probabilities
    log_pi_theta_w = F.log_softmax(base_w_logit, dim=-1).mean()
    log_pi_theta_l = F.log_softmax(base_l_logit, dim=-1).mean()
    log_pi_ref_w = F.log_softmax(ref_w_logit, dim=-1).mean()
    log_pi_ref_l = F.log_softmax(ref_l_logit, dim=-1).mean()

    # Compute DPO loss
    logits_diff = BETA * ((log_pi_theta_w - log_pi_ref_w) - (log_pi_theta_l - log_pi_ref_l))
    loss = -torch.log(torch.sigmoid(logits_diff)).mean()

    print("A6")
    return loss

# Optimizer
optimizer = torch.optim.Adam(base_model.parameters(), lr=1e-5)

# Training loop
EPOCHS = 3
for epoch in range(EPOCHS):
    total_loss = 0
    for _, row in df.iterrows():
        prompts = row["Question"]
        more_preferred = row["More_Prefered"]
        less_preferred = row["Less_Prefered"]

        # Compute loss
        loss = dpo_loss(base_model, ref_model, prompts, more_preferred, less_preferred)

        # Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print("A7")

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss / len(df)}")

# Save fine-tuned model
base_model.save_pretrained("gpt2-medium-dpo")
tokenizer.save_pretrained("gpt2-medium-dpo")

print("Training complete! Fine-tuned model saved.")


A1
A2
A3
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
Epoch 1/3, Loss: 0.6565073877573013
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
Epoch 2/3, Loss: 0.41382077187299726
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
A4
A5
A6
A7
Epoch 3/3, Loss: 0.30914057083427904
Training complete! Fine-tuned model saved.


In [ ]:
#TASK C

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=8b54e2fcda28a9147df301e0161ce8dbd2ca5f2b4092be674161b039bd684ae1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import evaluate

# Load BLEU and ROUGE metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Load fine-tuned models
rlhf_model = AutoModelForCausalLM.from_pretrained("gpt2-medium-rlhf").to("cuda")
dpo_model = AutoModelForCausalLM.from_pretrained("gpt2-medium-dpo").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

def generate_response(model, prompt):
    input_ids = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda")
    output_ids = model.generate(input_ids, max_length=100, do_sample=True, top_k=50, top_p=0.95, temperature=0.7)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Load test dataset
test_df = pd.read_csv("/kaggle/input/sample1/test_sample.csv")

rlhf_responses = []
dpo_responses = []
references = []

for _, row in test_df.iterrows():
    prompt = row["query"]
    reference = row["more_preferred"]

    rlhf_responses.append(generate_response(rlhf_model, prompt))
    dpo_responses.append(generate_response(dpo_model, prompt))
    references.append([reference.split()])  # BLEU expects list of tokenized references

# Compute BLEU scores
rlhf_bleu = bleu.compute(predictions=[r.split() for r in rlhf_responses], references=references)
dpo_bleu = bleu.compute(predictions=[r.split() for r in dpo_responses], references=references)

# Compute ROUGE scores (expects raw text, not tokenized lists)
rlhf_rouge = rouge.compute(predictions=rlhf_responses, references=[r[0] for r in references])
dpo_rouge = rouge.compute(predictions=dpo_responses, references=[r[0] for r in references])

# Print results
print(f"RLHF BLEU: {rlhf_bleu}")
print(f"DPO BLEU: {dpo_bleu}")
print(f"RLHF ROUGE: {rlhf_rouge}")
print(f"DPO ROUGE: {dpo_rouge}")

# Summarize findings
report = f"""
Performance Comparison: RLHF vs. DPO

Sample Efficiency:
- RLHF requires reward modeling and PPO training, which is computationally intensive.
- DPO uses direct preference optimization, avoiding explicit reward modeling, making it more sample-efficient.

Response Quality:
- RLHF BLEU: {rlhf_bleu}
- DPO BLEU: {dpo_bleu}
- RLHF ROUGE: {rlhf_rouge}
- DPO ROUGE: {dpo_rouge}

Computation Cost:
- RLHF involves complex PPO training with reward modeling, leading to higher computational overhead.
- DPO is relatively lightweight as it directly optimizes on preferred responses without reinforcement learning.

Conclusion:
DPO is more efficient and computationally cheaper, while RLHF can provide more controlled response shaping.
"""

print(report)


OSError: gpt2-medium-rlhf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import pandas as pd
from datasets import Dataset

# Set device dynamically (use GPU if available, otherwise use CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load fine-tuned models and tokenizer
# rlhf_model = AutoModelForCausalLM.from_pretrained("ppo_fine_tuned_model").to(device)
rlhf_model = AutoModelForCausalLM.from_pretrained("ppo_fine_tuned_model", ignore_mismatched_sizes=True).to(device)
dpo_model = AutoModelForCausalLM.from_pretrained("gpt2-medium-dpo").to(device)
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

# Fix: Set padding token for GPT-2 tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Load test dataset from CSV
test_dataframe = pd.read_csv("/kaggle/input/sample1/test_sample.csv")

# Convert to Hugging Face dataset and then a list
test_dataset = Dataset.from_pandas(test_dataframe, preserve_index=False).to_list()

# Define evaluation function
def evaluate_model(model, tokenizer, dataset):
    bleu_scores = []
    rouge_scores = {"rouge-1": [], "rouge-2": [], "rouge-l": []}
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    for entry in dataset:
        question = entry["Question"]  # Use "Question" as input
        reference_answer = entry["More_Prefered"]  # Use "More_Prefered" as ground truth

        # Tokenize and generate response
        input_ids = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
        generated_ids = model.generate(
            input_ids.input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id
        )
        generated_response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Compute BLEU score
        smoothie = SmoothingFunction().method4
        bleu = sentence_bleu([reference_answer.split()], generated_response.split(), smoothing_function=smoothie)
        bleu_scores.append(bleu)

        # Compute ROUGE scores
        rouge_results = scorer.score(reference_answer, generated_response)
        rouge_scores["rouge-1"].append(rouge_results["rouge1"].fmeasure)
        rouge_scores["rouge-2"].append(rouge_results["rouge2"].fmeasure)
        rouge_scores["rouge-l"].append(rouge_results["rougeL"].fmeasure)

    # Compute average scores
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge = {k: sum(v) / len(v) for k, v in rouge_scores.items()}

    return avg_bleu, avg_rouge

# Evaluate both models
print("Evaluating RLHF model...")
rlhf_bleu, rlhf_rouge = evaluate_model(rlhf_model, tokenizer, test_dataset)

print("Evaluating DPO model...")
dpo_bleu, dpo_rouge = evaluate_model(dpo_model, tokenizer, test_dataset)

# Print results
print("\nPerformance Comparison:")
print(f"RLHF BLEU Score: {rlhf_bleu:.4f}")
print(f"RLHF ROUGE Scores: {rlhf_rouge}")

print(f"DPO BLEU Score: {dpo_bleu:.4f}")
print(f"DPO ROUGE Scores: {dpo_rouge}")

# Save results to a file
with open("performance_comparison.txt", "w") as f:
    f.write("Performance Comparison:\n")
    f.write(f"RLHF BLEU Score: {rlhf_bleu:.4f}\n")
    f.write(f"RLHF ROUGE Scores: {rlhf_rouge}\n")
    f.write(f"DPO BLEU Score: {dpo_bleu:.4f}\n")
    f.write(f"DPO ROUGE Scores: {dpo_rouge}\n")

print("Evaluation complete! Results saved in performance_comparison.txt.")


Some weights of the model checkpoint at ppo_fine_tuned_model were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating RLHF model...
Evaluating DPO model...

Performance Comparison:
RLHF BLEU Score: 0.0572
RLHF ROUGE Scores: {'rouge-1': 0.17580831408775982, 'rouge-2': 0.010134876724203871, 'rouge-l': 0.09062154834822772}
DPO BLEU Score: 0.0563
DPO ROUGE Scores: {'rouge-1': 0.22199769053117785, 'rouge-2': 0.014775247953902247, 'rouge-l': 0.11602570539210763}
Evaluation complete! Results saved in performance_comparison.txt.
